# Skeleton of Assignment 4:

## The significant threshold for the whole exercise
$$ \alpha = 0.05 $$

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#imports downloader
from getCitiBikeCSV import getCitiBikeCSV
from __future__  import print_function
from scipy import stats
import os
#import for location process
import shapely.geometry

# 0. Read in data
I am reading in data from January 2015 with the function that I created getCitiBikeCSV. You are requested to use 2 months at least. It would be a good idea to use data from a colder and a warmer months, since there are more riders in the warm weather and ridership patterns may change with weather, temperature, etc. You should use data from multiple months, joining multiple datasets (thus addressing some systematic errors as well)

In [2]:
#download file of 2 months. one is colder, the other is warmer.
timeList = ['201601', '201608']
for datestring in enumerate (timeList):
    getCitiBikeCSV(datestring[1])

file in place, you can continue
file in place, you can continue


In [3]:
df1 = pd.read_csv(os.getenv("PUIDATA") + "/" + '201601' + '-citibike-tripdata.csv')
df8 = pd.read_csv(os.getenv("PUIDATA") + "/" + '201608' + '-citibike-tripdata.csv')

In [4]:
result = pd.concat([df1, df8], join='inner', ignore_index=True)
result.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,923,1/1/2016 00:00:41,1/1/2016 00:16:04,268,Howard St & Centre St,40.719105,-73.999733,3002,South End Ave & Liberty St,40.711512,-74.015756,22285,Subscriber,1958.0,1
1,379,1/1/2016 00:00:45,1/1/2016 00:07:04,476,E 31 St & 3 Ave,40.743943,-73.979661,498,Broadway & W 32 St,40.748549,-73.988084,17827,Subscriber,1969.0,1
2,589,1/1/2016 00:00:48,1/1/2016 00:10:37,489,10 Ave & W 28 St,40.750664,-74.001768,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,21997,Subscriber,1982.0,2
3,889,1/1/2016 00:01:06,1/1/2016 00:15:56,268,Howard St & Centre St,40.719105,-73.999733,3002,South End Ave & Liberty St,40.711512,-74.015756,22794,Subscriber,1961.0,2
4,1480,1/1/2016 00:01:12,1/1/2016 00:25:52,2006,Central Park S & 6 Ave,40.765909,-73.976342,2006,Central Park S & 6 Ave,40.765909,-73.976342,14562,Subscriber,1952.0,1


In [5]:
result.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender'],
      dtype='object')

In [6]:
result.drop(['stoptime', 'start station id', 'start station name', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype', 'gender'], axis=1, inplace=True)
result.head()

,tripduration,starttime,start station latitude,start station longitude,birth year
0,923,1/1/2016 00:00:41,40.719105,-73.999733,1958.0
1,379,1/1/2016 00:00:45,40.743943,-73.979661,1969.0
2,589,1/1/2016 00:00:48,40.750664,-74.001768,1982.0
3,889,1/1/2016 00:01:06,40.719105,-73.999733,1961.0
4,1480,1/1/2016 00:01:12,40.765909,-73.976342,1952.0


In [7]:
result['starttime'] = pd.to_datetime(result['starttime'], format='%m/%d/%Y %H:%M:%S')
result.head()

,tripduration,starttime,start station latitude,start station longitude,birth year
0,923,2016-01-01 00:00:41,40.719105,-73.999733,1958.0
1,379,2016-01-01 00:00:45,40.743943,-73.979661,1969.0
2,589,2016-01-01 00:00:48,40.750664,-74.001768,1982.0
3,889,2016-01-01 00:01:06,40.719105,-73.999733,1961.0
4,1480,2016-01-01 00:01:12,40.765909,-73.976342,1952.0


# Question 1: trip duration of bikers that ride during the day vs night
## 1.1 SPLIT BY CATEGORY

Splitting data by day and night at tripduration:

**H0: there is no statistical difference in the tripduration distribution of day and night trips**

In [8]:
timeuse = result.groupby(result['starttime'].dt.hour).count().loc[:,'start station latitude']
timeuse

starttime
0      18999
1      10713
2       6215
3       3918
4       3940
5      12180
6      47067
7      95668
8     169321
9     135799
10     92143
11     95370
12    105470
13    110103
14    113831
15    119359
16    142383
17    203334
18    198237
19    142666
20     94213
21     65115
22     48672
23     32425
Name: start station latitude, dtype: int64

Define the day is from 8a.m to 7p.m, night is from 8p.m to 7a.m.

In [9]:
dayHour = [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
nightHour = [0, 1, 2, 3, 4, 5, 6, 7, 20, 21, 22, 23]
dfDay = pd.DataFrame(columns=['tripduration', 'starttime',
                'start station latitude', 'start station longitude',
                'birth year'])
dfNight = pd.DataFrame(columns=['tripduration', 'starttime',
                'start station latitude', 'start station longitude',
                'birth year'])
for Hour in dayHour:
    dfDay = dfDay.append(result[result['starttime'].dt.hour==Hour])
for Hour in nightHour:
    dfNight = dfNight.append(result[result['starttime'].dt.hour==Hour])
len(dfDay)

1628016

In [10]:
len(dfNight)

439125

In [11]:
dfDay['tripduration'].dropna(inplace= True)
dfNight['tripduration'].dropna(inplace= True)

In [12]:
len(dfNight)

439125

In [13]:
#check if there are outliers
print ("the max value of dfDay is: {}".format(dfDay['tripduration'].max()))
print ("the median value of dfDay is: {}".format(dfDay['tripduration'].median()))
print ("Here are values of different percentage:\n{}".format(
    dfDay.tripduration.quantile([.25, .5, .75, .9, .95, .99, .998])))

the max value of dfDay is: 3966943
the median value of dfDay is: 643.0
Here are values of different percentage:
0.250      388
0.500      643
0.750     1099
0.900     1658
0.950     2052
0.990     3588
0.998    10850
Name: tripduration, dtype: object


There are some outliers at least above the percentange of 0.998.

In [ ]:
#detect and exclude outliers
dfDay = dfDay[(np.abs(stats.zscore(dfDay['tripduration'])) < 10)]
dfNight = dfNight[(np.abs(stats.zscore(dfNight['tripduration'])) < 10)]

print ("the max value of dfDay is: {}".format(dfDay['tripduration'].max()))
print ("quantile:\n{}".format(dfDay.tripduration.quantile([.25, .5, .75])))
print ("the max value of dfNight is: {}".format(dfNight['tripduration'].max()))
print ("quantile:\n{}".format(dfNight.tripduration.quantile([.25, .5, .75])))

the max value of dfDay is: 94743
quantile:
0.25     388
0.50     643
0.75    1099
Name: tripduration, dtype: object
the max value of dfNight is: 95627
quantile:
0.25    359
0.50    592
0.75    997
Name: tripduration, dtype: object


In [ ]:
fig = plt.figure(figsize=(10,30))
bins = np.arange(0, 6840, 342)
axDay = dfDay.tripduration.groupby(pd.cut(dfDay.tripduration, bins)).agg('count').plot(kind='bar', legend=False)
axDay.set_xlabel("Citibike trip duration")
axDay.set_ylabel("Number of trips")
axDay.set_title("Day trips")

bins = np.arange(0, 15431,)
axNight = dfNight.tripduration.groupby(pd.cut(dfNight.tripduration, bins)).agg('count').plot(kind='bar', legend=False)
axNight.set_xlabel("Citibike trip duration", fontsize = 18)
axNight.set_ylabel("Number of trips", fontsize = 18)
axNight.set_title("Night trips", fontsize = 20)

## 1.2 KS tests to compare 2 samples

http://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ks_2samp.html

the KS test in scipy returns the p-value BUT make sure you understand what the NULL is! read the documentation carefully! what is the null hypothesis that you can/cannot reject?

## 1.3 Pearson's  test for correlation

** notice that the Pearson's is a pairwise test: the samples need to be **
 a. the same size
 b. sorted! (how??)
    
http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html#scipy.stats.pearsonr



## 1.4 Spearman's  test for correlation

http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html#scipy.stats.spearmanr

** please comment on the numbers that you get in the light of the scipy manual: what is rerutned? what does it mean??**

# Question 2: age of bikers for trips originating in Manhattan and in Brooklyn
## 2.1 SPLIT BY CATEGORY

## 2.2 KS tests to compare 2 samples

## 2.3 Pearson's  test for correlation

## 2.4 Spearman's  test for correlation